In [34]:

def find_tree(messy_tree_df, parent_node='root', lv=1):
    """
    the nodes which become parent-less 
    after we remove the top level parent, 
    are the children of the removed parent
    """
    import pandas as pd
    new_tree = []
    all_nodes = set(messy_tree_df.child)
    nodes_have_true_parent = set(messy_tree_df[messy_tree_df.child!=messy_tree_df.parent].child)
    nodes_without_parent = all_nodes - nodes_have_true_parent
    for child in nodes_without_parent:
        new_tree.append((child,parent_node,lv))
    remaining_nodes = messy_tree_df[~messy_tree_df.child.isin(nodes_without_parent)]
    for node in list(nodes_without_parent):
        # remove the inode
        sub_tree = find_tree(remaining_nodes[remaining_nodes.parent!=node],node,lv+1)
        new_tree.extend(sub_tree)
    if lv!=1:
        return new_tree
    return pd.DataFrame(new_tree,columns=['child','parent','lv'])


In [40]:
tree = [(8,8),(9,9),(10,10),(11,11),
    (6,7),    (1,6),    (2,6),    (3,6),    (0,6),    (4,6),    
    (7,7),    (6,6),    (1,1),    (2,2),    (3,3),    (0,0),    (4,4),      (5,5),  
    (1,7),    (2,7),    (3,7),    (0,7),    (4,7),    (5,4),       (5,6),       (5,7),    
]
import pandas as pd
tree = pd.DataFrame(tree, columns=['child','parent'])


In [41]:
find_tree(tree)

,child,parent,lv
0,8,root,1
1,9,root,1
2,10,root,1
3,11,root,1
4,7,root,1
5,6,7,2
6,0,6,3
7,1,6,3
8,2,6,3
9,3,6,3


In [42]:
def find_direct_parent(tree, node, direct_parents):
    if node in direct_parents:
        return direct_parents[node]
    node_parents = tree[tree.child==node].parent.values
    if len(node_parents)==0:
        direct_parents[node]=(node, 'root',1)
        return direct_parents[node]
    node_parents_lv = [find_direct_parent(tree, p, direct_parents)[2] for p in node_parents]
    max_lv = max(node_parents_lv)
    direct_parent = node_parents[node_parents_lv.index(max_lv)]
    direct_parents[node] = (node, direct_parent, max_lv+1)
    return (node, direct_parent, max_lv+1)

def find_tree2(messy_tree_df):
    direct_parents={}
    all_nodes = set(messy_tree_df.child)
    messy_tree_df = messy_tree_df[messy_tree_df.child!=messy_tree_df.parent].copy()
    new_tree = []
    for node in all_nodes:
        x = find_direct_parent(messy_tree_df, node, direct_parents)
        new_tree.append(x)
    return pd.DataFrame(new_tree,columns=['child','parent','lv']).sort('lv')



,child,parent,lv
7,7,root,1
8,8,root,1
9,9,root,1
10,10,root,1
11,11,root,1
6,6,7,2
0,0,6,3
1,1,6,3
2,2,6,3
3,3,6,3


In [43]:
%timeit find_tree(tree)
%timeit find_tree2(tree)

100 loops, best of 3: 13.9 ms per loop
100 loops, best of 3: 5.95 ms per loop


In [31]:
max([1,2,3])
[1,2,3].index(3)

2